In [1]:

import numpy as np
from numba import njit
import edlib


def get_rc(s):
    map_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    l = []
    for c in s:
        l.append(map_dict[c])
    l = l[::-1]
    return ''.join(l)
def rc(s):
    map_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    l = []
    for c in s:
        l.append(map_dict[c])
    l = l[::-1]
    return ''.join(l)

def seq2hashtable_multi_test(refseq, testseq, kmersize=15, shift = 1):
    rc_testseq = get_rc(testseq)
    testseq_len = len(testseq)
    local_lookuptable = dict()
    skiphash = hash('N'*kmersize)
    for iloc in range(0, len(refseq) - kmersize + 1, 1):
        hashedkmer = hash(refseq[iloc:iloc+kmersize])
        if(skiphash == hashedkmer):
            continue
        if(hashedkmer in local_lookuptable):

            local_lookuptable[hashedkmer].append(iloc)
        else:
            local_lookuptable[hashedkmer] = [iloc]
    iloc = -1
    readend = testseq_len-kmersize+1
    one_mapinfo = []
    preiloc = 0
    while(True):
   
        iloc += shift
        if(iloc >= readend):
            break

        #if(hash(testseq[iloc: iloc + kmersize]) == hash(rc_testseq[-(iloc + kmersize): -iloc])):
            #continue
 
        hashedkmer = hash(testseq[iloc: iloc + kmersize])
        if(hashedkmer in local_lookuptable):

            for refloc in local_lookuptable[hashedkmer]:

                one_mapinfo.append((iloc, refloc, 1, kmersize))



        hashedkmer = hash(rc_testseq[-(iloc + kmersize): -iloc])
        if(hashedkmer in local_lookuptable):
            for refloc in local_lookuptable[hashedkmer]:
                one_mapinfo.append((iloc, refloc, -1, kmersize))
        preiloc = iloc

    

    return np.array(one_mapinfo)

def get_points(tuples_str):
    data = []
    num = 0
    for c in tuples_str:
        if(ord('0') <= c <= ord('9')):
            num = num * 10 + c - ord('0')
        elif(ord(',') == c):
            data.append(num)
            num = 0
    if(num != 0):
        data.append(num)
    return data

def calculate_distance(ref, query, ref_st, ref_en, query_st, query_en):
    A = ref[ref_st: ref_en]
    a = query[query_st: query_en]
    _a = rc(query[query_st: query_en])
    return min(edlib.align(A, a)['editDistance'], edlib.align(A, _a)['editDistance'])

def get_first(x):
    return x[0]


def calculate_value(tuples_str, ref, query):  

    slicepoints = np.array(get_points(tuples_str.encode()))
    if(len(slicepoints) > 0 and len(slicepoints) % 4 == 0):
        editdistance = 0
        aligned = 0
        preend = 0
        points = np.array(slicepoints).reshape((-1, 4)).tolist()
        points.sort(key=get_first)
        for onetuple in points:
            query_st, query_en, ref_st, ref_en = onetuple
            if(preend > query_st):
                return 0
            if(query_en - query_st < 30):
                continue
            preend = query_en
            if((calculate_distance(ref, query, ref_st, ref_en, query_st, query_en)/len(query[query_st:query_en])) > 0.1):
                continue
            editdistance += calculate_distance(ref, query, ref_st, ref_en, query_st, query_en)
            aligned += len(query[query_st:query_en])
        return max(aligned - editdistance, 0)
    else:
        return 0


In [2]:
#实验1
ref = '...'
query = '...'


In [3]:
data = seq2hashtable_multi_test(ref, query, kmersize=9, shift = 1)
data.shape

(63746, 4)

In [4]:
#实验2
ref = 'TGATTTAGAACGGACTTAGCAGACATTGAAACTCGAGGGGTATAGCAATAGATGCCCAAAAAGGTAAGCGCCATAAGCGTGGTTCTACGAGCCAGGTGCTCATGCCTAAGTTCTGCGCCTTCGCTGTCACTTGGAAATACTGTAATGGATCATGCCTAGGTTATGCGCCTTCGGGGTCACTTCAACATACTGTAATGGATCATGCCTAGGTTTTGCGTGTTCGCTGTCATTTCGAAATACTCCAATGGATGATGCCTAGGTTCTGTGCCTTCGCTGACGCATGGAAATACTTTAACGGATCATGCCCAGGCTCTGCGCCTTCGCTGAAACTTCGAAATACTCTAATGGATCATGCCTCGGTGCTCCACCTTCGCTTTCATTCCGAAATACTCTAATGGATCGCGTCCGTGTAACAACTTCGTACTGTTATATCGACCATCAGAATACCCATCCCTCGGGGAGGTAACCTATATTCACGTCGCAAGTTTCGATCTACAGTATGCTGACTGTTTGCCGCGATTTTAAGTCAAGAAGCGCGTCCACATGGTCGCGGTCGTCAACTTCAGTACGCTCATATGACACCAAAAGATCTACCTACAGCCCGTGCAGCTCGACTTTTGTGCTCTAGGGCACGACGGGTGGCGTTTGCTCCCGCGCATCTCGACTTTTAAGCTCTATGGCACAACGTGTGGCGTTTGCCCCCGCGCAGCTCGACTTTTGTGCTCTAGGGCACGGCGGGTGGCGTTTGCCCTCGCCCAGCTTGACTTTTGTGCTCTAGGGCACGACGGGTGGCGTTTGCCCCCGTGCAGCCCGACTTTTGTACTCTAGTGCACGACGGGTGGCGTTTGCCCCCGCACCGCTCGACTTTTGTGATCTAGGGCACTACGAGTAGCGTTGGCCCAGACAGATCAACGCACATGGATTCTCTAACAGGCCCCGCGCTTCTCATTGGCCCGTGAGACGGGTCTGAGAGGAAGACATTAGGTAGATACGGAAAGCTTTGGCGTAGTTCGTATCTTTCAGCGGTGAAGCGTCTTCGGTCCGGGCTGCGTTATGCCTGCGGGAGGAAGGCTCCACTAGATGGTTTACGAGACATAATGTCAGCTTCCTAAAGGTACGGCAGCGCCTGCGTATATCACAGGACGAATTGTCAGTTTGCTAGGGGTACGGGAGCGCTTGCGTATTACATAGGACGAATCGTCAGCTTCCTAAAGGGACGGTAGCGCTTGCGTGTTACATAGGACGAATTGTCAGCTTCGTAAAGGTACGGTAGTTCTTGCGTATTACATAGGATGCATTGTCCGCTTCCTAAAGGTACGCTGGCGCTTGCGTATCACATAGGACGGATAGCGCGATTGCTAAAGGTACGGGAGCGCTTGCGTCTTAGAGCGCACGAATCGGATATAAGCTGCGCCCGCGTCTGGCGAGCAAAAATCGTGGGAGCCAGCGAGGGAAAAACTGCTCGGGCGACTTAAACGGAATTACAAGACTCATTGCCATCGAGGACGTTAGACTAAAGAGCCCCTGCGTGCCTCCTTTGTATAGCTCGATGTAGTGGCCCGTGTATGTGGAACAGGAATGCTCGATCTAAGGTAGTAGTGGCTACAGCTCCGAGAGTTTGCGTACTGCGGTGCCAGGGATTTTGCCTGCGGGTACAGCCTCTGCGCACGCCGGTCTGTGATCAAGAACTAAACTAGAGA'
query = 'TGATTTAGAACGGACTTAGCAGACATTGAAACTCGAGGGGTATAGCAATAGATGCCCAAAAAGGTAAGCGCCATAAGCGTGTTTCTACGAGCCAGGTGCTCATGCCTAAGTTCTGCGCCTTCGCTGTCACTGGGAAATACTGTAATGGATCATCCGTAGGTTATGCGCCTTCGGGGTCACTTCAACATACTGTAACGGATCGTGCCTAGGTTTTGCGTATTCGCTGTCATTTCGAATTACACCAATGGATGATGCCTAGGTTCTGTGCCTCCGCTGACGCATCGAAATACTTTAACGGATCGCGTCCGAGTAACAACTTCGTACTGTTATATAGGCAATCAGAATACCCATGCCTCGGGGAGGTAACCTATATTCACGTCGCAAGTTTCGATCTACAGTACTGTAGGTATATCTTTTGGTGTCATATGAGGGTACTGAACTTGACGACCGCGACCATGTGGATGCGCTTCTTGACTTAAAATCGCGGCAAACAGTAAGCATCCGTGAAGCTCGACTTTTGTGCTCTAGGGCACGACGGGTGGCGTTTGCTCCCGCGCATCTCGAGTTGTAAGCTCTATGGCACAACGGGTGGCGTTTGCCGCCGAGCAGCTCGACTTTTGTGCTCTAGGGCACGGCGGGTGGCGTTTGCCCTCGCCCAGCTTGACTTTTGTGCTCTAGGGCACGACGGGTGGCCTTTGCCCCCGCGCAGCTGGACTTTTGTGCTCTAGGGCACGGCGGGTGGCGTTTGCCCTCCCCCAGCTTGACTACTGTGCTCTAGGGCACGACGGGTGGCGTTTGCCCCCGCGCAGCTCGACTTTTGTGCTCTATGGCACGGGGGGTGGCGTTTGCCCTCGCCCAGCTTGACTTTTGCGCTCTAGGGCACGACGGGTGGCGTTTGCCGGCAAACGCCACACGTCGTGCCCTAGAGCACAAACGTCAAGCTGGGCGAGGGCAACCGCCACCCGCCCTGCCCTAGAGCACAAAAGTCGAGCTGCGCGGGCCCGCGCAGCTCGACTTTTGTGCTCTAGGACACGGCGGGTGGCGTTTGCCCTCGCCCAGCTTGACTCTTGTGCTCTAGGGCACGACGGGTGGCGTTTGCCCCAGCGCAGCCCGACTTTTGTACTCTAGAGCACGACGGTTGGCATTTGCCCCCGCACCGCTCGACTTTTGTGATCTAGGGCCCTAGGAGTAGCGTTGGCCAGCTTTCCGTATCTACCTAATGTCTTCCTCTCAGACCCGTCTCACGGGCCAATGAGAAGCGCGGGGCCTATTAGAGAATCCATGTGCGTTGATCTGTCTGCAGACAGCTCAACGCACATGGATTCGCTAGCAGGCCCCGCGCTTCTCATTGGCCCGTGAGACGGGTCTGAGAGGAAGACATAAGGTAGATACGGCAAGCTCACGTCCGTGTAACAACGTCGTACTGTTATATCGACCATCAGAATCCCCATCCCGCGAGGAGGTAACCTATATTCAGGTCGCAAGTTTCGATCTACAGTATTGGCGTAGTTCGTATCTTTCAGCGGTGAAGCTTCTTCGGTCCGGGCTGCGTTATGCCTGCTGGAGGACGGCTCCACTAGATGGTTTACGAGACATAATGTCCGCTTCCTAAAGGTACACTGGCGCTTGAGTATCACATAGGACGGATAGCTCGATTCCTAAAGGGACGGGAGCGCTTGCGTCTTAGAGCGCATGAATCGTCAGCTTCCCAAAGGGACCGTAGCGCTTGCGTGTTATATAGGAAGAATGGTCAGCTTTGTAAAGGTACGGTAGTTCTTGCGTATTACAGAGGATGCATTGTCTACTACCTAAAGGTACGGCAGCGCCTGCGTATATCACAGGACGAATTGTCAGTTTGCTAGGGGTACGGGAGCGCTTGCATATTACATAGGACGAATCGGATATAAGCTGCGCCCGCGTCTGGCGATAAAAAATCGTGGTAGCCAGCGAGGGAAAAACTGCTCGGGCGACTTAAACGGAATTAAAAGACTCATTGCCGTGACAGACTTCCGTATAGCAACCTCTGGGATGTCGATGCGGTGTCCCCAGTCTGCGCTGAGCGGGGGCAGACAGACTTAGTTATAGTATGCATCTGTTTTAGCTAGACATCACGACCTAGTGGGGTTCATGTTGAGATTCTAGGGCGGTACGCAGCCGGTGGATTATTACTTCCCCAGAAATTCTGACTTCGTCACTGGATGGATTGTACTATCCGGTCAACCTTACAAGGTTTCAACAGGGACGAAGGGTAAACGTATGAAGCTTGGATGCCGTTACCGTAAAGGGCCCTATTGAAGTGTCGAGGACGTTAGACTAAAGAGCCCCTGCGTGCCTCCTTTGTATAGCTCGAGGTAGTGGCCCGGATATGTGGAACAGGAATGCTCGATCTAAGGTAGTAGTGGGTACCGCTCCGAGAGTTTGCGTACTGCGGTGCCCGGGATTTTGCCTGCGGGTACAGCCTCTGCGCACGCCGGTCTGTAATCAAGAACTAAACTAGAGA'


In [5]:
data = seq2hashtable_multi_test(ref, query, kmersize=9, shift = 1)
data.shape

(2347, 4)

在这里设计你的算法

In [ ]:
import sys
from collections import defaultdict, namedtuple
import bisect

# Define a seed match record
Seed = namedtuple('Seed', ['q_pos', 'r_pos'])
def build_kmer_index(ref: str, k: int) -> dict:
    index = defaultdict(list)
    for i in range(len(ref) - k + 1):
        kmer = ref[i:i+k]
        index[kmer].append(i)
    return index

def find_seeds(query: str, ref_index: dict, k: int) -> list:
    """
    Find all exact k-mer matches between query and reference.
    Returns list of Seed(q_pos, r_pos).
    """
    seeds = []
    for i in range(len(query) - k + 1):
        kmer = query[i:i+k]
        for r_pos in ref_index.get(kmer, []):
            seeds.append(Seed(q_pos=i, r_pos=r_pos))
    return seeds


def chain_seeds(seeds: list, max_gap: int = 50) -> list:
    """
    Chain seeds using improved DP with binary search over r_pos to speed up transitions.
    Optimized to O(n log n) using patience sorting-style structure.
    """
    if not seeds:
        return []

    # Sort by query and reference positions
    seeds.sort(key=lambda s: (s.q_pos, s.r_pos))

    # Each chain ends at some seed with increasing q_pos and r_pos
    dp = []  # Stores last r_pos of chains
    chain_ends = []  # Stores the actual seeds in the best chains
    parents = [-1] * len(seeds)
    indices = []  # Stores the index of the seed ending each chain

    for i, s in enumerate(seeds):
        pos = bisect.bisect_left(dp, s.r_pos)
        if pos == len(dp):
            dp.append(s.r_pos)
            indices.append(i)
        elif s.r_pos < dp[pos]:
            dp[pos] = s.r_pos
            indices[pos] = i

        if pos > 0:
            parents[i] = indices[pos - 1]

    # Reconstruct the best chain
    last = indices[-1]
    chain = []
    while last != -1:
        chain.append(seeds[last])
        last = parents[last]
    chain.reverse()
    return chain


def merge_chain(chain: list, k: int) -> list:
    """
    Merge adjacent seeds in chain into longer segments.
    Returns list of (q_start, q_end, r_start, r_end).
    """
    if not chain:
        return []

    segments = []
    start = chain[0]
    q_start, r_start = start.q_pos, start.r_pos
    prev = start

    for s in chain[1:]:
        if s.q_pos == prev.q_pos + 1 and s.r_pos == prev.r_pos + 1:
            prev = s
            continue
        segments.append((q_start, prev.q_pos + k, r_start, prev.r_pos + k))
        q_start, r_start = s.q_pos, s.r_pos
        prev = s

    segments.append((q_start, prev.q_pos + k, r_start, prev.r_pos + k))
    return segments


def align(query: str, ref: str, k: int = 11, max_gap: int = 50) -> list:
    """
    High-level function to compute alignment segments between query and reference.
    """
    ref_index = build_kmer_index(ref, k)
    seeds = find_seeds(query, ref_index, k)
    chain = chain_seeds(seeds, max_gap)
    segments = merge_chain(chain, k)
    return segments

Result

In [ ]:
tuples_str = str(youfunction(data))

In [8]:
#Score
calculate_value(tuples_str, ref, query)

2090